In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import vectorbt as vbt
import yfinance as yf
from talipp.indicators import  RSI
import pandas_ta as ta
import plotly
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def get_stock_data_with_daily_emas(symbol, start_date, end_date):
    try:
        dfdaily= yf.download(symbol, start=start_date, end=end_date, interval='1d', progress=False)
        dfdaily=dfdaily.reset_index()
        dfdaily.columns = [x.lower() for x in dfdaily.columns]
        # Calculate emas
        dfdaily["ema50"] = dfdaily['close'].ewm(span=50, adjust=False, min_periods=12).mean()
        
        # Initialize columns for signal
        dfdaily['signal'] = False
        return dfdaily
    
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return None

In [6]:
def calculate_max_drawdown(values):
    """
    Calculate the maximum drawdown percentage from a list or array of values.
    Returns:
    - max_drawdown_percentage: Maximum drawdown percentage.
    """
    mdd = 0
    peak = values[0]
    for x in values:
        if x > peak: 
            peak = x
        dd = (peak - x) / peak
        if dd > mdd:
            mdd = dd
    return round(mdd*100,2)


In [7]:
np.seterr(divide='ignore', invalid='ignore')
def calculate_max_drawdown_duration(values):
    """
    Calculate the maximum drawdown duration from a list or array of values.
    Returns:
    - max_drawdown_duration: Maximum drawdown duration (in periods).
    """
    mdd_duration = 0
    dd=0
    peak = values[0]
    for x in values:
        if x > peak: 
            peak = x
            dd=0
        elif x < peak:
            dd=dd+1
        if dd > mdd_duration:
            mdd_duration = dd
    return mdd_duration 

In [ ]:
def simulate_investing( stock_data):
    num_shares = 0
    final_balance = 0
    invested_money = 0
    stock_data['equity'] = ''
    stock_data['pnl'] = ''
    stock_data['invested_money'] = ''
    # Simulate buying and selling based on the generated signals
    #length of data 
    for i in range(len(stock_data)):
        if stock_data['signal'][i] == True:
            num_shares = num_shares + (stock_data['regular_investment'][i] / stock_data['close'][i] ) 
            invested_money  = invested_money + stock_data['regular_investment'][i]
        stock_data['equity'][i] = num_shares*stock_data['close'][i]    
        stock_data['invested_money'][i] = invested_money  
        stock_data['pnl'][i] = stock_data['equity'][i]-stock_data['invested_money'][i]
    # Calculate the final balance based on the last stock price and remaining number of shares
    total_return = (stock_data['equity'][len(stock_data['equity'])-1] - invested_money) / invested_money * 100
    # Return the final balance, profit/loss, total return, and total trades
    return {
        'Final Balance': stock_data['equity'][len(stock_data['equity'])-1],
        'Profit/Loss':  stock_data['equity'][len(stock_data['equity'])-1]-invested_money,
        'Invested Money':  invested_money,
        'Total Return': total_return,
        'Max Strategy Drawdown [%]': calculate_max_drawdown(stock_data['equity']) ,
        'Max Strategy Drawdown Duration(Days)': calculate_max_drawdown_duration(stock_data['equity']) ,	
        'Benchmark Return':  (stock_data['close'][len(stock_data['close'])-1] -stock_data['close'][0]) / stock_data['close'][0] * 100,
        'Max Benchmark Drawdown [%]': calculate_max_drawdown(stock_data['close']) ,
        'Max Benchmark Drawdown Duration(Days)': calculate_max_drawdown_duration(stock_data['close']) ,	
        'Number of trading days': len(stock_data)
    }


In [ ]:
def calculate_average(row):
    # Convert the values in the row to numeric, ignoring non-numeric values
    numerical_values = pd.to_numeric(row, errors='coerce')
    
    # Check if there are numerical values in the row
    if not numerical_values.empty:
        # Calculate the mean of the numerical values
        return np.mean(numerical_values)
    
    # If no numerical values, return NaN
    return np.nan

In [2]:
def calculate_portfolio_statistics(tickers,generate_buy_above_ema_signal,start_date,end_date):
    # Initialize an empty list to store the results for each ticker
    results = []

    # Loop through each ticker in the list of tickers
    for ticker in tickers:
        # Initialize a list to store temporary statistics for the current ticker
        temp_stats = []
        data = generate_buy_above_ema_signal(
            get_stock_data_with_daily_emas(symbol=ticker, start_date=start_date, end_date=end_date)
        )
        # Apply functions to calculate portfolio statistics for the current ticker
        temp_stats = simulate_investing( data)

        # Create a DataFrame from the temporary statistics
        df = pd.DataFrame(temp_stats, index=[0])
        df = pd.melt(df)
        df = df.set_index(df.iloc[:, 0]).iloc[:, 1:]
        df.index.name = None
        df = df.rename(columns={df.columns[0]: ticker})

        # Append the DataFrame to the list of results
        results.append(df)

    # Concatenate the list of DataFrames into a final DataFrame
    final_df = pd.concat(results, axis=1)

    # Apply rounding to two decimal places
    final_df = final_df.round(2)
    final_df['average'] = final_df.apply(calculate_average, axis=1)
    final_df = final_df.round(2)
    return final_df
